# Interface testing
## 1. Interface definition: Model

Code error building strings in common/make_aux_dirs

In [4]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from abc import ABC, abstractmethod
import pathlib
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
print("GPU Available: ", tf.test.is_gpu_available())
print("Eager execution enabled: ", tf.executing_eagerly())

GPU Available:  False
Eager execution enabled:  True


In [47]:

class Model(ABC):

    def __init__(self, name, path, input_dim, output_dim):
        '''
        Initialization function of the class Model. Parameters needed are:
        \t- name: (string) name of the model, models directory name
        \t- data_dir: (path) where the data directory is located
        \t- models_dir: (path) where the models directory is located
        \t- input_dim: (int) input dimension, must be multiple of 32
        \t- output_dim: (int) the number of classes to train
        \t- path: (Path) working directory to store everything
        '''
        self.models = {}  # paths included data_dir : Path ; models_dir : Path
        self.data = {}
        self.test_data = np.empty([output_dim, input_dim, 1, 1])
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.__name = name
        self.models['data_dir'] = path / 'test_data'
        if not os.path.exists(self.models['data_dir']):
            self.models['data_dir'].mkdir()
        self.models['models_dir'] = path / 'models'
        if not os.path.exists(self.models['models_dir']):
            self.models['models_dir'].mkdir()

    @property
    def name(self):
        return self.__name

    @name.setter
    def setname(self, name):
        self.__name = name

    @name.getter
    def getname(self):
        return self.__name

    @abstractmethod
    def save_model_to_file():
        '''
        Function to store training and original model files in the
        corresponding format.
        '''
        pass

    @abstractmethod
    def build(self):
        # kb arguments for the compiler?
        # instantiate model object
        # polimorphism argmax
        '''
        Here should be the model definition to be built,
        compiled and summarized.The model should be stored
        in the dictionary with its name: self.models[self.name]=model
        '''
        pass

    @abstractmethod
    def load(self, load_path):  # restore Models state with submodels?
        '''
        If we don't want to build our model from scratch and
        we have it stored somewhere, we can load it with this function.
        - load_path: path where the model is stored
        '''
        pass

    @abstractmethod
    def prep_data(self):  # Loading and preprocessing
        # everything that doesn't happend on the fly
        '''
        To prepare or download the training and test data.
        Should return a dictionary:
        {'x_train':xt, 'y_train':yt, 'x_test':xtt, 'y_test':ytt}
        '''
        pass

    @abstractmethod
    def train(self):
        '''
        GPU and CPU usage should be differentiated.
        Fit with hyperparams and if we want to save
        original model and training data,
        that should be done here.
        '''
        pass

    @abstractmethod
    def gen_test_data(self):  # naming
        '''
        Select the test data examples for storing
        along with the converted models.
        Must fill the data dictionary with an entry called 'export_data'
        '''
        pass

    # REDO from here down bc of polimorphy ~~

    @abstractmethod
    def to_tf_float(self):  # polymorphism affects here
        '''
        Create converter from original model
        to TensorFlow Lite Float.
        Converter stored with the key 'model_float'.
        '''
        assert self.name in self.models

    @abstractmethod
    def to_tf_quant(self):
        '''
        Create converter from original model
        to TensorFlow Lite with quantization.
        Converter stored with the key 'model_quant'.
        '''
        assert self.name in self.models

    @abstractmethod
    def to_tf_stripped(self):
        '''
        Conversion from quantized model
        to TensorFlow Lite with quantization
        and stripped of float in/out tensors.
        Stored with the key 'model_stripped'.
        '''
        assert 'model_quant' in self.models

    @abstractmethod
    def to_tf_xcore(self):
        '''
        Conversion from the quantized model
        to TensorFlow Lite with optimizations
        for the xcore ai.
        Stored with the key 'model_xcore'.
        '''
        assert 'model_quant' in self.models

    def populate_converters(self):
        '''
        Create all the converters in a row in the logical order.
        The only thing needed is the presence
        of the original model in the models dictionary:
        self.models[self.name] must exist.
        '''
        assert self.name in self.models
        self.to_tf_float()
        self.to_tf_quant()
        self.to_tf_stripped()
        self.to_tf_xcore()

    @abstractmethod
    def convert_and_save(self):
        '''
        Will save all the models in the self.models dictionary along with the
        test data provided as parameter.
        The models to be saved are:
        \t- tflite float
        \t- tflite quant
        \t- tflite stripped
        \t- tflite xcore
        '''
        test_data = self.data['export_data']
        # float
        model_float_file = common.save_from_tflite_converter(
            self.models['model_float'],
            self.models['models_dir'],
            "model_float")
        common.save_test_data_for_regular_model(
            model_float_file,
            test_data,
            data_dir=self.models['data_dir'],
            base_file_name="model_float")
        # quant
        model_quant_file = common.save_from_tflite_converter(
            self.models['model_quant'],
            self.models['models_dir'],
            "model_quant")
        common.save_test_data_for_regular_model(
            model_quant_file,
            test_data,
            data_dir=self.models['data_dir'],
            base_file_name="model_quant")
        # stripped
        common.save_from_json(self.models['model_stripped'],
                              self.models['models_dir'],
                              'model_stripped')
        common.save_test_data_for_stripped_model(
            self.models['model_stripped'],
            test_data,
            data_dir=self.models['data_dir'])
        # xcore
        common.save_from_json(self.models['model_xcore'],
                              self.models['models_dir'],
                              'model_xcore')
        common.save_test_data_for_xcore_model(
            self.models['model_xcore'],
            test_data,
            data_dir=self.models['data_dir'])

## 2. Polymorphism
### 2.1 Keras

In [48]:

class KerasModel(Model):

    @abstractmethod
    def build(self):
        pass

    @abstractmethod
    def prep_data(self):
        pass

    @abstractmethod
    def train(self, BS, EPOCHS):
        assert self.data
        self.models[self.name].fit(
            self.data['x_train'],
            self.data['y_train'],
            epochs=EPOCHS,
            batch_size=BS,
            validation_data=(self.data['x_test'], self.data['y_test']))
        # save model and data
        self.save_model_to_file()

    @abstractmethod
    def gen_test_data(self):
        '''
        self.data['export_data'] =
        '''
        pass

    def save_model_to_file(self):
        np.savez(self.models['data_dir'] / 'training_data', **self.data)
        self.models[self.name].save(str(self.models['models_dir']/'model.h5'))

    def load(self):
        train_path = self.models['data_dir']/'training_data.npz'
        model_path = self.models['models_dir']/'model.h5'
        try:
            logging.info(f"Loading data from {train_path}")
            self.data = dict(np.load(train_path))
            logging.info(f"Loading keras model from {model_path}")
            self.models[self.name] = tf.keras.models.load_model(model_path)
        except FileNotFoundError as e:
            logging.error(f"{e} (Hint: use the --train_model flag)")
            return
        out_shape = self.models[self.name].output_shape[1]
        if out_shape != self.output_dim:
            raise ValueError(f"number of specified classes ({self.output_dim})"
                             f"does not match model output shape ({out_shape})"
                             )

    def to_tf_float(self):  # affected by poly
        assert self.name in self.models
        self.models['model_float'] = tf.lite.TFLiteConverter.from_keras_model(
            self.models[self.name])

    def to_tf_quant(self):  # affected by poly
        assert self.name in self.models
        assert 'x_train' in self.data
        self.models['model_quant'] = tf.lite.TFLiteConverter.from_keras_model(
            self.models[self.name])
        common.quantize_converter(
            self.models['model_quant'], self.data['x_train'])

    def to_tf_stripped(self):  # not really affected by poly
        assert 'model_quant' in self.models
        desc = "TOCO Converted and stripped."
        model_quant_file = common.save_from_tflite_converter(
            self.models['model_quant'],
            self.models['models_dir'],
            "model_quant")
        model_quant = tflite_utils.load_tflite_as_json(model_quant_file)
        self.models['model_stripped'] = common.strip_model_quant(model_quant)
        self.models['model_stripped']['description'] = desc

    def to_tf_xcore(self):  # not really affected by poly
        assert 'model_quant' in self.models
        self.models['model_xcore'] = deepcopy(self.models['model_quant'])
        graph_conv.convert_model(self.models['model_xcore'],
                                 remove_softmax=True)

    def convert_and_save(self):
        super().convert_and_save()

### 2.2 SavedModel

In [4]:

class SavedModel(Model):

    @abstractmethod
    def build(self):
        pass

    @abstractmethod
    def load(self, load_path):
        pass

    @abstractmethod
    def prep_data(self):
        pass

    @abstractmethod
    def train(self):
        pass

    @abstractmethod
    def gen_test_data(self):
        pass

### 2.3 FunctionModel

In [5]:

class FunctionModel(Model):

    @abstractmethod
    def build(self):
        pass

    @abstractmethod
    def load(self, load_path):
        pass

    @abstractmethod
    def prep_data(self):
        pass

    @abstractmethod
    def train(self):
        pass

    @abstractmethod
    def gen_test_data(self):
        pass

# DEMO: Interface implementation: fc_deepin_shallowout_final -- refactor

In [49]:
import examples_common as common

import os
import argparse
import logging
import pathlib
from pathlib import Path
import tflite_utils

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

import tflite2xcore_graph_conv as graph_conv
from copy import deepcopy

from tflite2xcore_utils import clean_unused_buffers, clean_unused_tensors
from tflite2xcore_utils import XCOps


class FcDeepinShallowoutFinal(KerasModel):

    # add keyboard optimizer, loss and metrics???
    def build(self, input_dim, out_dim=2):
        input_dim = self.input_dim
        output_dim = self.output_dim
        # Env
        tf.keras.backend.clear_session()
        tflite_utils.set_all_seeds()
        # Building
        model = tf.keras.Sequential(name=self.name)
        model.add(layers.Flatten(input_shape=(input_dim, 1, 1),
                                 name='input'))
        model.add(layers.Dense(output_dim, activation='softmax',
                               name='ouptut'))
        # Compilation
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        # Add to dict
        self.models[self.name] = model
        # Show summary
        model.summary()

    def prep_data(self):
        self.data = generate_fake_lin_sep_dataset(
            self.output_dim,
            self.input_dim,
            train_samples_per_class=51200//self.output_dim,
            test_samples_per_class=10240//self.output_dim)

    def gen_test_data(self):
        if not self.data:
            self.prep_data()
        subset_inds = np.searchsorted(self.data['y_test'].flatten(),
                                      np.arange(self.output_dim))
        self.data['export_data'] = self.data['x_test'][subset_inds]

    def train(self):
        self.BS = 128
        self.EPOCHS = 5*(self.output_dim-1)
        super().train(self.BS, self.EPOCHS)

## Data generation function

In [50]:

def generate_fake_lin_sep_dataset(classes=2, dim=32, *,
                                  train_samples_per_class=5120,
                                  test_samples_per_class=1024):
    z = np.linspace(0, 2*np.pi, dim)

    # generate data and class labels
    x_train, x_test, y_train, y_test = [], [], [], []
    for j in range(classes):
        mean = np.sin(z) + 10*j/classes
        cov = 10 * np.diag(.5*np.cos(j * z) + 2) / (classes-1)
        x_train.append(
            np.random.multivariate_normal(
                mean, cov, size=train_samples_per_class))
        x_test.append(
            np.random.multivariate_normal(
                mean, cov, size=test_samples_per_class))
        y_train.append(j * np.ones((train_samples_per_class, 1)))
        y_test.append(j * np.ones((test_samples_per_class, 1)))

    # stack arrays
    x_train = np.vstack(x_train)
    y_train = np.vstack(y_train)
    x_test = np.vstack(x_test)
    y_test = np.vstack(y_test)

    # normalize
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std

    # expand dimensions for TFLite compatibility
    def expand_array(arr):
        return np.reshape(arr, arr.shape + (1, 1))
    x_train = expand_array(x_train)
    x_test = expand_array(x_test)

    return {'x_train': np.float32(x_train), 'y_train': np.float32(y_train),
            'x_test': np.float32(x_test), 'y_test': np.float32(y_test)}


## Instantiate the model and build

In [51]:
# !rm -rf test_data
# !rm -rf models
test_model = FcDeepinShallowoutFinal(
    'fc_deepin_shallowout_final', Path('.'), 32, 2)
print(test_model.models)

{'data_dir': PosixPath('test_data'), 'models_dir': PosixPath('models')}


In [52]:
print('Model name property:', test_model.name)
test_model.build(32)  # breaks with sequencial syntax

Model name property: fc_deepin_shallowout_final
Model: "fc_deepin_shallowout_final"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Flatten)              (None, 32)                0         
_________________________________________________________________
ouptut (Dense)               (None, 2)                 66        
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


## Data generation and train

In [53]:
test_model.prep_data()
print(len(test_model.data))
test_model.data.keys()

4


dict_keys(['x_train', 'y_train', 'x_test', 'y_test'])

In [54]:
test_model.train()

Train on 51200 samples, validate on 10240 samples
Epoch 1/5
51200/51200 [==============================] - 0s 9us/sample - loss: 0.1268 - accuracy: 0.9546 - val_loss: 0.0397 - val_accuracy: 0.9909
Epoch 2/5
51200/51200 [==============================] - 0s 6us/sample - loss: 0.0265 - accuracy: 0.9944 - val_loss: 0.0202 - val_accuracy: 0.9956
Epoch 3/5
51200/51200 [==============================] - 0s 6us/sample - loss: 0.0158 - accuracy: 0.9966 - val_loss: 0.0141 - val_accuracy: 0.9968
Epoch 4/5
51200/51200 [==============================] - 0s 6us/sample - loss: 0.0117 - accuracy: 0.9972 - val_loss: 0.0111 - val_accuracy: 0.9978
Epoch 5/5
51200/51200 [==============================] - 0s 6us/sample - loss: 0.0095 - accuracy: 0.9976 - val_loss: 0.0093 - val_accuracy: 0.9979


### If not training, loading the model

In [55]:
test_model.load()
print(test_model.data.keys())
print(test_model.models.keys())
print(test_model.models['fc_deepin_shallowout_final'])

dict_keys(['x_train', 'y_train', 'x_test', 'y_test'])
dict_keys(['data_dir', 'models_dir', 'fc_deepin_shallowout_final'])


## Generate test data

In [56]:
test_model.gen_test_data()

## Conversions

In [57]:
test_model.to_tf_float()

In [58]:
test_model.to_tf_quant()

In [59]:
test_model.to_tf_stripped()

In [61]:
# Checks
print(test_model.data.keys())
print(test_model.models.keys())

dict_keys(['x_train', 'y_train', 'x_test', 'y_test', 'export_data'])
dict_keys(['data_dir', 'models_dir', 'fc_deepin_shallowout_final', 'model_float', 'model_quant', 'model_stripped'])


In [62]:
test_model.to_tf_xcore()  # broken

TypeError: can't pickle _thread.RLock objects

In [18]:
test_model.populate_converters()

TypeError: can't pickle _thread.RLock objects

In [63]:
test_model.convert_and_save()

RuntimeError: tensorflow/lite/kernels/kernel_util.cc:119 std::abs(input_product_scale - bias_scale) <= 1e-6 * std::min(input_product_scale, bias_scale) was not true.Node number 1 (FULLY_CONNECTED) failed to prepare.
